## 라이브러리

In [10]:
import pandas as pd
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

## 1. 데이터 로드

In [11]:
# CSV 파일 읽기
products_df = pd.read_csv('쿠키_data.csv')
products_df['full_description'] = products_df['title'].fillna('') + products_df['info_text'].fillna('') + products_df['detail_paragraph'].fillna('')
products_df

,상품url,asin,title,brand,original_price,discounted_price,avg_rating,rating_count,info_text,detail_paragraph,...,category_3,category_4,imgUrl_1,imgUrl_2,imgUrl_3,imgUrl_4,imgUrl_5,imgUrl_6,imgUrl_7,full_description
0,https://www.amazon.com/-/ko/dp/B000FNFL0G/ref=...,B000FNFL0G,"Royal Dansk 덴마크 쿠키 셀렉션, 방부제 또는 색소 첨가 없음, 355ml...",Royal Dansk,567,6731,4.6,87417.0,바삭하고 버터 쿠키 클래식: 이 355ml(12온스) 틴 안에 풍부한 덴마크 버터 ...,1966년부터 Royal Dansk 는 덴마크에서 수입된 정통 덴마크 버터 쿠키로 ...,...,쿠키,버터,https://m.media-amazon.com/images/I/718tQBoi7U...,https://m.media-amazon.com/images/I/51z94Swf7l...,https://m.media-amazon.com/images/I/71o0ZSgMS-...,https://m.media-amazon.com/images/I/818nIAq27-...,https://m.media-amazon.com/images/I/61TMS4+HNV...,https://m.media-amazon.com/images/I/51Vf+MPT3L...,https://m.media-amazon.com/images/I/71hUqEl93I...,"Royal Dansk 덴마크 쿠키 셀렉션, 방부제 또는 색소 첨가 없음, 355ml..."
1,https://www.amazon.com/-/ko/dp/B006SOI7Y8/ref=...,B006SOI7Y8,"Nyakers 진저브레드 스냅 쿠키 틴, 최고급 진저 스냅 오리지널 맛 스웨덴 쿠키...",Nyakers,1662,43627,4.5,2848.0,진짜 생강으로 제작 전통적인 스웨덴 생강 스냅 쿠키는 배송 중에 보호하기 위해 보호...,Nyåkers 생강 스냅은 전통적인 스웨덴 생강 스냅과 같은 맛입니다. Nyåker...,...,쿠키,진저 스냅,https://m.media-amazon.com/images/I/81i8IpgxfH...,https://m.media-amazon.com/images/I/616LXsPDkU...,https://m.media-amazon.com/images/I/81d8R7w0N4...,https://m.media-amazon.com/images/I/813vW+NLjj...,https://m.media-amazon.com/images/I/81oyAnFnpW...,https://m.media-amazon.com/images/I/61HrYTSuof...,NaN,"Nyakers 진저브레드 스냅 쿠키 틴, 최고급 진저 스냅 오리지널 맛 스웨덴 쿠키..."
2,https://www.amazon.com/-/ko/dp/B098QZF1FJ/ref=...,B098QZF1FJ,"Simple Mills Sweet Thins 쿠키, 씨앗 및 견과류 가루, 민트 초...",Simple Mills,15789,15789,4.5,7189.0,"영양이 풍부한 수박 씨앗 가루, 캐슈 가루, 해바라기 씨 가루 및 아마를 포함한 실...",Simple Mills 간단하고 맛있는 진짜 음식을 통해 삶과 몸을 풍부하게 합니다...,...,쿠키,초콜릿,https://m.media-amazon.com/images/I/81qmPKuemQ...,https://m.media-amazon.com/images/I/81ZMhTJ8z1...,https://m.media-amazon.com/images/I/81ZnSO5phR...,https://m.media-amazon.com/images/I/81eMJXC61v...,https://m.media-amazon.com/images/I/81lk4Im8s3...,https://m.media-amazon.com/images/I/81QVIzvWrd...,NaN,"Simple Mills Sweet Thins 쿠키, 씨앗 및 견과류 가루, 민트 초..."
3,https://www.amazon.com/-/ko/dp/B0872CS81R/ref=...,B0872CS81R,"Dewey's Bakery Meyer 레몬 쿠키 씬 | 인공 향료, 합성 색소 또는...",Dewey's Bakery,1204,10803,4.0,473.0,Meyer 레몬 쿠키 1박스가 포함되어 있습니다. 각 255.1g(9온스) 상자에는...,NaN,...,쿠키,쿠키 부류,https://m.media-amazon.com/images/I/81CmXK+0L0...,https://m.media-amazon.com/images/I/819109AHPI...,https://m.media-amazon.com/images/I/916HKE+DLh...,https://m.media-amazon.com/images/I/91XoIohJ2i...,https://m.media-amazon.com/images/I/91JlUF5nPT...,https://m.media-amazon.com/images/I/91MW2zJtB9...,NaN,"Dewey's Bakery Meyer 레몬 쿠키 씬 | 인공 향료, 합성 색소 또는..."
4,https://www.amazon.com/-/ko/dp/B07BS3BGNH/ref=...,B07BS3BGNH,"Dewey's Bakery 브라우니 크리스프 쿠키 씬 | 인공 향료, 합성 색소 또...",Dewey's Bakery,1218,10927,4.2,611.0,브라우니 크리스프 쿠키 1박스가 포함되어 있습니다. 각 255.1g(9온스) 상자에...,NaN,...,쿠키,초콜릿,https://m.media-amazon.com/images/I/81MED1XiPg...,https://m.media-amazon.com/images/I/81LYFNor5n...,https://m.media-amazon.com/images/I/91Ftb96mwS...,https://m.media-amazon.com/images/I/81CNx0BSwQ...,https://m.media-amazon.com/images/I/81LYFNor5n...,https://m.media-amazon.com/images/I/91XoIohJ2i...,https://m.media-amazon.com/images/I/A1RnmGLbEK...,"Dewey's Bakery 브라우니 크리스프 쿠키 씬 | 인공 향료, 합성 색소 또..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,https://www.amazon.com/-/ko/dp/B003A9Q7E6/ref=...,B003A9Q7E6,"Pirouline 롤 웨이퍼 - 다크 초콜릿 - 롤 웨이퍼 스틱, 크림 충전 웨이퍼...",Pirouline,1204,47076,4.3,2667.0,특허받은 시그니처 소용돌이 - Pirouline 롤 웨이퍼에서만 찾을 수 있습니다....,NaN,...,쿠키,웨이퍼,https://m.media-amazon.com/images/I/819+gVcW+9...,https://m.media-amazon.com/images/I/71tka7hGWu...,https://m.media-amazon.com/images/I/711ZQvFupS...,https://m.media-amazon.com/images/I/81E1wROkCc...,https://m.media-amazon.com/images/I/71ta6PEmBd...,https://m.media-amazon.com/images/I/71ugM7HVB6...,NaN,"Pirouline 롤 웨이퍼 - 다크 초콜릿 - 롤 웨이퍼 스틱, 크림 충전 웨이퍼..."
114,https://www.amazon.com/-/ko/dp/B0B29G49

## 2. 함수 정의

In [12]:
# 이모티콘 제거 함수
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"  # other pictographs 
                               u"\U0001F926-\U0001F937"
                               u"\U00010000-\U0010ffff"
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [13]:
# 불용어 파일을 불러오는 함수
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return f.read().splitlines()

# 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', text)  # 특수 문자, 숫자 제거
    return re.sub(r'\s+', ' ', text).strip()  # 불필요한 공백 제거

# 형태소 분석 및 키워드 추출 함수
def extract_keywords(text, stopwords):
    okt = Okt()
    words = okt.pos(text, stem=True)
    return [word[0] for word in words if word[1] in ['Noun', 'Adjective'] and word[0] not in stopwords]

# TF-IDF 기반 키워드 추출 함수
def extract_keywords_tfidf(text, stopwords, top_n=10, threshold=0.2):
    try:
        tfidf = TfidfVectorizer(tokenizer=lambda x: extract_keywords(x, stopwords))
        tfidf_matrix = tfidf.fit_transform([text])
        feature_names = tfidf.get_feature_names_out()
        scores = tfidf_matrix.toarray()[0]
        sorted_indices = scores.argsort()[::-1][:top_n]
        return [feature_names[idx] for idx in sorted_indices if scores[idx] > threshold]
    except ValueError as e:
        print(f"오류 발생: {e}. 문서를 건너뜁니다.")
        return []

# 전체 텍스트 전처리 함수
def process_text(text, stopwords):
    text = remove_emojis(text)
    text = clean_text(text)
    return text

## 3. 불용어 파일 로드 및 키워드 추출 실행

In [14]:
# 불용어 로드
stopwords_filepath = 'stopword.txt'
stopwords = load_stopwords(stopwords_filepath)

# 각 텍스트에 대해 전처리 및 키워드 추출
products_df['cleaned_description'] = products_df['full_description'].apply(lambda x: process_text(x, stopwords))
products_df['keyword'] = products_df['cleaned_description'].apply(lambda x: extract_keywords_tfidf(x, stopwords, top_n=10, threshold=0.1))

C:\Users\yejis\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


오류 발생: empty vocabulary; perhaps the documents only contain stop words. 문서를 건너뜁니다.
오류 발생: empty vocabulary; perhaps the documents only contain stop words. 문서를 건너뜁니다.


In [15]:
# 빈 리스트를 가진 행을 삭제
products_df = products_df[products_df['keyword'].apply(lambda x: len(x) > 0)].reset_index(drop=True)
products_df

,상품url,asin,title,brand,original_price,discounted_price,avg_rating,rating_count,info_text,detail_paragraph,...,imgUrl_1,imgUrl_2,imgUrl_3,imgUrl_4,imgUrl_5,imgUrl_6,imgUrl_7,full_description,cleaned_description,keyword
0,https://www.amazon.com/-/ko/dp/B000FNFL0G/ref=...,B000FNFL0G,"Royal Dansk 덴마크 쿠키 셀렉션, 방부제 또는 색소 첨가 없음, 355ml...",Royal Dansk,567,6731,4.6,87417.0,바삭하고 버터 쿠키 클래식: 이 355ml(12온스) 틴 안에 풍부한 덴마크 버터 ...,1966년부터 Royal Dansk 는 덴마크에서 수입된 정통 덴마크 버터 쿠키로 ...,...,https://m.media-amazon.com/images/I/718tQBoi7U...,https://m.media-amazon.com/images/I/51z94Swf7l...,https://m.media-amazon.com/images/I/71o0ZSgMS-...,https://m.media-amazon.com/images/I/818nIAq27-...,https://m.media-amazon.com/images/I/61TMS4+HNV...,https://m.media-amazon.com/images/I/51Vf+MPT3L...,https://m.media-amazon.com/images/I/71hUqEl93I...,"Royal Dansk 덴마크 쿠키 셀렉션, 방부제 또는 색소 첨가 없음, 355ml...",덴마크 쿠키 셀렉션 방부제 또는 색소 첨가 없음 온스바삭하고 버터 쿠키 클래식 이 ...,"[쿠키, 버터, 덴마크, 맛, 스타일, 바삭, 첨가, 코코넛, 클래식, 바닐라]"
1,https://www.amazon.com/-/ko/dp/B006SOI7Y8/ref=...,B006SOI7Y8,"Nyakers 진저브레드 스냅 쿠키 틴, 최고급 진저 스냅 오리지널 맛 스웨덴 쿠키...",Nyakers,1662,43627,4.5,2848.0,진짜 생강으로 제작 전통적인 스웨덴 생강 스냅 쿠키는 배송 중에 보호하기 위해 보호...,Nyåkers 생강 스냅은 전통적인 스웨덴 생강 스냅과 같은 맛입니다. Nyåker...,...,https://m.media-amazon.com/images/I/81i8IpgxfH...,https://m.media-amazon.com/images/I/616LXsPDkU...,https://m.media-amazon.com/images/I/81d8R7w0N4...,https://m.media-amazon.com/images/I/813vW+NLjj...,https://m.media-amazon.com/images/I/81oyAnFnpW...,https://m.media-amazon.com/images/I/61HrYTSuof...,NaN,"Nyakers 진저브레드 스냅 쿠키 틴, 최고급 진저 스냅 오리지널 맛 스웨덴 쿠키...",진저브레드 스냅 쿠키 틴 최고급 진저 스냅 오리지널 맛 스웨덴 쿠키 온스 파운드 보...,"[스냅, 생강, 스웨덴, 보호, 쿠키, 맛, 제작, 전통, 인서트]"
2,https://www.amazon.com/-/ko/dp/B098QZF1FJ/ref=...,B098QZF1FJ,"Simple Mills Sweet Thins 쿠키, 씨앗 및 견과류 가루, 민트 초...",Simple Mills,15789,15789,4.5,7189.0,"영양이 풍부한 수박 씨앗 가루, 캐슈 가루, 해바라기 씨 가루 및 아마를 포함한 실...",Simple Mills 간단하고 맛있는 진짜 음식을 통해 삶과 몸을 풍부하게 합니다...,...,https://m.media-amazon.com/images/I/81qmPKuemQ...,https://m.media-amazon.com/images/I/81ZMhTJ8z1...,https://m.media-amazon.com/images/I/81ZnSO5phR...,https://m.media-amazon.com/images/I/81eMJXC61v...,https://m.media-amazon.com/images/I/81lk4Im8s3...,https://m.media-amazon.com/images/I/81QVIzvWrd...,NaN,"Simple Mills Sweet Thins 쿠키, 씨앗 및 견과류 가루, 민트 초...",쿠키 씨앗 및 견과류 가루 민트 초콜릿 글루텐 프리 팔레오 친화적 건강한 간식 온스...,"[음식, 가루, 영양, 풍부하다, 씨앗, 삶, 맛있다, 몸, 수박, 맛]"
3,https://www.amazon.com/-/ko/dp/B0872CS81R/ref=...,B0872CS81R,"Dewey's Bakery Meyer 레몬 쿠키 씬 | 인공 향료, 합성 색소 또는...",Dewey's Bakery,1204,10803,4.0,473.0,Meyer 레몬 쿠키 1박스가 포함되어 있습니다. 각 255.1g(9온스) 상자에는...,NaN,...,https://m.media-amazon.com/images/I/81CmXK+0L0...,https://m.media-amazon.com/images/I/819109AHPI...,https://m.media-amazon.com/images/I/916HKE+DLh...,https://m.media-amazon.com/images/I/91XoIohJ2i...,https://m.media-amazon.com/images/I/91JlUF5nPT...,https://m.media-amazon.com/images/I/91MW2zJtB9...,NaN,"Dewey's Bakery Meyer 레몬 쿠키 씬 | 인공 향료, 합성 색소 또는...",레몬 쿠키 씬 인공 향료 합성 색소 또는 방부제 없음 소량으로 구운 온스 팩 레몬 ...,"[쿠키, 색소, 합성, 레몬, 인공, 방부제, 향료, 홈, 듀이, 마법]"
4,https://www.amazon.com/-/ko/dp/B07BS3BGNH/ref=...,B07BS3BGNH,"Dewey's Bakery 브라우니 크리스프 쿠키 씬 | 인공 향료, 합성 색소 또...",Dewey's Bakery,1218,10927,4.2,611.0,브라우니 크리스프 쿠키 1박스가 포함되어 있습니다. 각 255.1g(9온스) 상자에...,NaN,...,https://m.media-amazon.com/images/I/81MED1XiPg...,https://m.media-amazon.com/images/I/81LYFNor5n...,https://m.media-amazon.com/images/I/91Ftb96mwS...,https://m.media-amazon.com/images/I/81CNx0BSwQ...,https://m.media-amazon.com/images/I/81LYFNor5n...,https://m.media-amazon.com/images/I/91XoIohJ2i...,https://m.media-amazon.com/images/I/A1RnmGLbEK...,"Dewey's Bakery 브라우니 크리스프 쿠키 씬 | 인공 향료, 합성 색소 또...",브라우니 크리스프 쿠키 씬 인공 향료 합성 색소 또는 방부제 없음 소량으로 구운 온...,"[쿠키, 합성, 스프, 크리, 향료, 브라우니, 색소, 인공, 방부제, 홈]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,https://www.amazon.com/-/ko/dp/B00R2O4YJU/ref=...,B00R2O4YJU,"OREO 레몬 크림 샌드위치 쿠키, 432.3g(15.5온스)",Oreo,1274,19376,4.5,2179.0,Oreo 레몬 크림 샌드위치 쿠키 432.3g(15.5온스) 팩 1개 맛있는 레몬 ..

In [16]:
first_row_keyword = products_df.loc[0, 'keyword']
print("첫 번째 행의 keyword 값:", first_row_keyword)

첫 번째 행의 keyword 값: ['쿠키', '버터', '덴마크', '맛', '스타일', '바삭', '첨가', '코코넛', '클래식', '바닐라']


## 4. CSV 파일 저장

In [17]:
#최종 결과를 CSV로 저장
products_df.to_csv('붕대 및 붕대용품_full.csv', index=False, encoding='utf-8-sig')